In [1]:
# from image_processing import *
import dlib
import cv2
import numpy as np

# Load the detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [2]:
def extract_eye_region(image, landmarks, eye_points, buffer= 1):
    # Extract the coordinates of the eye points
    region = np.array([(landmarks.part(point).x, landmarks.part(point).y) for point in eye_points])
    # Create a mask with zeros
    height, width = image.shape[:2]
    mask = np.zeros((height, width), np.uint8)
    # Fill the mask with the polygon defined by the eye points
    cv2.fillPoly(mask, [region], 255)
    # Bitwise AND operation to isolate the eye region
    eye = cv2.bitwise_and(image, image, mask=mask)
    # Cropping the eye region
    (min_x, min_y) = np.min(region, axis=0)
    (max_x, max_y) = np.max(region, axis=0)
    min_x = max(min_x - buffer, 0)
    min_y = max(min_y - buffer, 0)
    max_x = min(max_x + buffer, width)
    max_y = min(max_y + buffer, height)
    cropped_eye = eye[min_y:max_y, min_x:max_x]
    return cropped_eye, (min_x, min_y, max_x, max_y)


In [3]:
import cv2

def enhance_image_resolution(image):
    # Load the super-resolution model
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    path = "EDSR_x4.pb"  # Change to the path of the model
    sr.readModel(path)
    sr.setModel("edsr", 4)  # You can change the model and scale as needed

    # Enhance the resolution of the image
    enhanced_image = sr.upsample(image)
    return enhanced_image


In [4]:
def visualize_image(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [5]:
def is_pupil_candidate(contour, gray_eye_image, eye_center, threshold=0.7):
    #show gray_eye_image

    # Calculate area and perimeter of the contour
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    print("area: ", area)
    print("perimeter: ", perimeter)

    # Check if perimeter is zero to avoid division by zero
    if perimeter == 0:
        print("Perimeter is zero")
        return False

    # Check for circularity
    circularity = 4 * np.pi * (area / (perimeter * perimeter))
    print("circularity: ", circularity)
    if circularity < threshold:
        return False

    # Check aspect ratio of bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h
    print("aspect_ratio: ", aspect_ratio)
    if aspect_ratio < 0.8 or aspect_ratio > 1.2:
        return False

    # Check solidity
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    solidity = float(area) / hull_area
    print("solidity: ", solidity)
    if solidity < threshold:
        return False

    # Check relative location to the center of the eye
    M = cv2.moments(contour)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    distance_to_center = np.sqrt((cX - eye_center[0])**2 + (cY - eye_center[1])**2)
    # if distance_to_center > (w ):
    #     print("distance_to_center: ", distance_to_center)
    #     return False

    # Check intensity
    mask = np.zeros(gray_eye_image.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_val = cv2.mean(gray_eye_image, mask=mask)[0]
    if mean_val > 50:  # This threshold can be adjusted based on your images
        print("mean_val: ", mean_val)
        return False

    return True

In [6]:
def detect_pupil(eye_image, eye_center):
    # Enhance resolution
    eye_image = enhance_image_resolution(eye_image)

    gray = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
    visualize_image("Grayscale Eye", gray)  # Visualize grayscale eye

    gray = cv2.equalizeHist(gray)  # Histogram Equalization
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Gaussian Blur
    visualize_image("Blurred Eye", blurred)  # Visualize blurred eye

    # Find contours
    contours, _ = cv2.findContours(blurred, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    # Sort contours by area and filter
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Visualize contours
    contour_image = eye_image.copy()
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
    visualize_image("Contours", contour_image)

    for contour in contours:

        # if is_pupil_candidate(contour, gray, eye_center):
            # Found a good pupil candidate
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.circle(eye_image, (cX, cY), 7, (255, 255, 255), -1)
            visualize_image("Pupil", eye_image)
            return (cX, cY), contour

    print("No suitable pupil found")
    return None, None

In [7]:
def pre_process_image(image):        
    # Initialize variables
    left_eye_info = right_eye_info = left_eye_bbox = right_eye_bbox = None

    dlib_faces = detector(image)
    processed_data = []
    for dlib_face in dlib_faces:
        shape = predictor(image, dlib_face)

        for (i, (start, end)) in enumerate([(36,42), (42,48)]):
            eye_landmarks = [(shape.part(point).x, shape.part(point).y) for point in range(start, end)]
            eye_center = np.mean(eye_landmarks, axis=0).astype(int)  # Ensure you have integers for the center
            eye_image, (eye_min_x, eye_min_y, eye_max_x, eye_max_y) = extract_eye_region(image, shape, range(start, end))
  
            pupil_center, pupil_contour = detect_pupil(eye_image, eye_center)
            print(pupil_center)
            # After detecting the pupil in the cropped eye image:
            if pupil_center:
                # Scale the pupil center coordinates down to the original image size
                pupil_center_original = (pupil_center[0] / 4, pupil_center[1] / 4)
                # Transform these coordinates to the global space of the original image
                pupil_center_global = (int(pupil_center_original[0]) + eye_min_x, int(pupil_center_original[1]) + eye_min_y)
    
                pupil_center_global = tuple(pc.item() if isinstance(pc, np.generic) else pc for pc in pupil_center_global)

                #draw contours
                cv2.drawContours(image, [pupil_contour], -1, (0, 255, 0), 2)
                cv2.circle(image, pupil_center_global, 1, (255, 255, 255), -1)
                cv2.imshow("Eye", image)
                cv2.waitKey(0)
                cv2.destroyAllWindows()


                bounding_box = (eye_min_x, eye_min_y, eye_max_x - eye_min_x, eye_max_y - eye_min_y)
                bounding_box = tuple(bb.item() if isinstance(bb, np.generic) else bb for bb in bounding_box)

                eye_data = {
                    'eye_position': 'left' if i == 0 else 'right',
                    'pupil_center': pupil_center_global,
                    'bounding_box': bounding_box
                }
                processed_data.append(eye_data)

        # Processed data for each eye
    for eye_data in processed_data:
        if eye_data['eye_position'] == 'left':
            left_eye_info = eye_data['pupil_center']
            left_eye_bbox = eye_data['bounding_box']
        else:
            right_eye_info = eye_data['pupil_center']
            right_eye_bbox = eye_data['bounding_box']
    
    # Check if any eye information was detected
    if left_eye_info is None and right_eye_info is None:
        print("No eye information detected")
        return None

    return processed_data, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, shape
    

In [8]:
image = 'data/Naia/calibration_images/Naia_07779f08-1c1f-490f-9d26-7786c43aca1d.png'
pre_process_image(cv2.imread(image))
# # Load your image here
# image = cv2.imread(image)
# eye_image = extract_eye_region(image)


(73, 25)
(68, 24)


([{'eye_position': 'left',
   'pupil_center': (310, 166),
   'bounding_box': (292, 160, 38, 12)},
  {'eye_position': 'right',
   'pupil_center': (394, 170),
   'bounding_box': (377, 164, 35, 12)}],
 (310, 166),
 (394, 170),
 (292, 160, 38, 12),
 (377, 164, 35, 12),
 <_dlib_pybind11.full_object_detection at 0x157fffaa9b0>)

In [115]:
image1 = 'data/Will/eye_gaze_images/Will_0c426e4f-72ac-479c-a1e6-3d9df004aebb.png'
pre_process_image(cv2.imread(image1))

(65, 27)
(59, 25)


([{'eye_position': 'left',
   'pupil_center': (288, 233),
   'bounding_box': (272, 227, 33, 13)},
  {'eye_position': 'right',
   'pupil_center': (364, 237),
   'bounding_box': (350, 231, 30, 12)}],
 (288, 233),
 (364, 237),
 (272, 227, 33, 13),
 (350, 231, 30, 12),
 <_dlib_pybind11.full_object_detection at 0x25a8f2395b0>)

In [110]:
image2 = 'data/muzzy/calibration_images/muzzy_0194b294-0505-498e-afe8-837a4e1b5cf6.png'
pre_process_image(cv2.imread(image2))

(82, 30)
(83, 30)


([{'eye_position': 'left',
   'pupil_center': (294, 225),
   'bounding_box': (274, 218, 43, 14)},
  {'eye_position': 'right',
   'pupil_center': (394, 227),
   'bounding_box': (374, 220, 43, 14)}],
 (294, 225),
 (394, 227),
 (274, 218, 43, 14),
 (374, 220, 43, 14),
 <_dlib_pybind11.full_object_detection at 0x25a8f2e34f0>)